In [1]:
import os
os.chdir('/home/sankar/VSCODE_PROJECTS/Medical-Detection3d-Toolkit')
import pandas as pd
import SimpleITK as sitk
import shutil
import numpy as np
import json
from gen_landmark_mask import gen_single_landmark_mask

/home/sankar/VSCODE_PROJECTS/Medical-Detection3d-Toolkit


In [2]:
os.getcwd()

'/home/sankar/VSCODE_PROJECTS/Medical-Detection3d-Toolkit'

Setting up the target and root path

In [3]:
root_path = './assets/trial/case2'
target_path = './assets/data_set/'

# Dataset Structure

The dataset is organized in the following folder structure:


In [4]:
# data_set
# ├── patient_id1
# │   ├── patient_id1_landmark_mask.nii.gz
# │   ├── patient_id1.csv
# │   └── patient_id1.nii.gz
# ├── patient_id2
# │   ├── patient_id2_landmark_mask.nii.gz
# │   ├── patient_id2.csv
# │   └── patient_id2.nii.gz
# ├── patient_id3
# ├── patient_id..
# ├── patient_idn

In [5]:
landmark_label = {
                     'Go': 1,
                     'Me': 2,
                     'H': 3,
                  }
pos_upper_bound = 3
neg_lower_bound = 6
target_image_origin = (0,0,0)

for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
      image_dir= os.path.join(target_path, os.path.basename(dirpath))
      print(image_dir)
      if filename.endswith('.json'):
         landmark_json_path = os.path.join(dirpath,filename)

         filenames.pop(filenames.index(filename))

         if filenames != []:
            for file in filenames:
               if file.endswith('.nii.gz'):
                  image_volume_path = os.path.join(dirpath,'volume.nii.gz')
                  image_volume = sitk.ReadImage(image_volume_path)
                  origin = image_volume.GetOrigin()

                  image_volume.SetOrigin((0,0,0))

                  spacing = image_volume.GetSpacing()

                  with open(landmark_json_path, 'r') as file:
                     landmark_json = json.load(file)
                  
                  landmark_cos = []
                  for control_point in landmark_json['markups'][0]['controlPoints']:
                     landmark_co = control_point['position']
                     landmark_co = np.round(np.array(landmark_co) - np.array(origin),decimals=1)
                     landmark_cos.append({"name":control_point['label'], "x":landmark_co[0], "y":landmark_co[1], "z":landmark_co[2]})

                  landmark_cos_df = pd.DataFrame(landmark_cos)
                  
                  landmark_mask = gen_single_landmark_mask(image_volume, landmark_cos_df, landmark_label, spacing, pos_upper_bound, neg_lower_bound)
                  has_negative_values = (landmark_cos_df[['x', 'y', 'z']] < 0).any().any()
                  if (image_volume.GetOrigin() == target_image_origin and 
                                landmark_mask.GetOrigin() == target_image_origin and 
                                has_negative_values!=True):
                     os.makedirs(image_dir, exist_ok=True)

                     landmark_csv_path = os.path.join(image_dir ,os.path.basename(dirpath)+'.csv')
                     image_volume_path = os.path.join(image_dir,os.path.basename(dirpath)+'.nii.gz')
                     landmark_path = os.path.join(image_dir, os.path.basename(dirpath)+'_landmark_mask.nii.gz')
                     
                     landmark_cos_df.to_csv(landmark_csv_path, index=False)
                     sitk.WriteImage(image_volume, image_volume_path, useCompression=True)
                     sitk.WriteImage(landmark_mask, landmark_path, useCompression=True)
                  else:
                     print('IMage origin is not 0,0,0 or landmarks have -ve values')

               else:
                  print("----VOLUME FILE MISSING.----")
         else:
            print("----VOLUME FILE MISSING.----")
      else:
         print("----Landmark FILE MISSING.----")

./assets/data_set/S0000000007
./assets/data_set/S00000000010
./assets/data_set/S0000000004
./assets/data_set/S0000000001
----Landmark FILE MISSING.----
./assets/data_set/S0000000009
./assets/data_set/S0000000003
./assets/data_set/S0000000008
./assets/data_set/S0000000002
----VOLUME FILE MISSING.----
./assets/data_set/S0000000006
./assets/data_set/S0000000005


Generating train and test .csv file

In [6]:
path_to_data ={'image_name': [], 'image_path': [], 'landmark_file_path': [], 'landmark_mask_path':[]}

for dirpath, dirnames, filenames in os.walk(target_path):
    if filenames != []:
        print(dirpath, dirnames, filenames)
        path_to_data['image_name'].append(os.path.basename(dirpath))
        for filename in filenames:
            if filename.endswith('.nii.gz'):           
                if filename.endswith('mask.nii.gz'):
                    path_to_data['landmark_mask_path'].append(os.path.join(dirpath, filename))
                else:
                    path_to_data['image_path'].append(os.path.join(dirpath, filename))

            if filename.endswith('.csv'):
                path_to_data['landmark_file_path'].append(os.path.join(dirpath, filename))

./assets/data_set/S0000000007 [] ['S0000000007_landmark_mask.nii.gz', 'S0000000007.csv', 'S0000000007.nii.gz']
./assets/data_set/S00000000010 [] ['S00000000010.nii.gz', 'S00000000010.csv', 'S00000000010_landmark_mask.nii.gz']
./assets/data_set/S0000000004 [] ['S0000000004.csv', 'S0000000004.nii.gz', 'S0000000004_landmark_mask.nii.gz']
./assets/data_set/S0000000009 [] ['S0000000009.csv', 'S0000000009_landmark_mask.nii.gz', 'S0000000009.nii.gz']
./assets/data_set/S0000000003 [] ['S0000000003.csv', 'S0000000003.nii.gz', 'S0000000003_landmark_mask.nii.gz']
./assets/data_set/S0000000008 [] ['S0000000008.csv', 'S0000000008_landmark_mask.nii.gz', 'S0000000008.nii.gz']
./assets/data_set/S0000000006 [] ['S0000000006.csv', 'S0000000006.nii.gz', 'S0000000006_landmark_mask.nii.gz']
./assets/data_set/S0000000005 [] ['S0000000005.csv', 'S0000000005.nii.gz', 'S0000000005_landmark_mask.nii.gz']


In [8]:
data_df = pd.DataFrame(path_to_data)

In [9]:
data_df

,image_name,image_path,landmark_file_path,landmark_mask_path
0,S0000000007,./assets/data_set/S0000000007/S0000000007.nii.gz,./assets/data_set/S0000000007/S0000000007.csv,./assets/data_set/S0000000007/S0000000007_land...
1,S00000000010,./assets/data_set/S00000000010/S00000000010.ni...,./assets/data_set/S00000000010/S00000000010.csv,./assets/data_set/S00000000010/S00000000010_la...
2,S0000000004,./assets/data_set/S0000000004/S0000000004.nii.gz,./assets/data_set/S0000000004/S0000000004.csv,./assets/data_set/S0000000004/S0000000004_land...
3,S0000000009,./assets/data_set/S0000000009/S0000000009.nii.gz,./assets/data_set/S0000000009/S0000000009.csv,./assets/data_set/S0000000009/S0000000009_land...
4,S0000000003,./assets/data_set/S0000000003/S0000000003.nii.gz,./assets/data_set/S0000000003/S0000000003.csv,./assets/data_set/S0000000003/S0000000003_land...
5,S0000000008,./assets/data_set/S0000000008/S0000000008.nii.gz,./assets/data_set/S0000000008/S0000000008.csv,./assets/data_set/S0000000008/S0000000008_land...
6,S0000000006,./assets/data_set/S0000000006/S0000000006.nii.gz,./assets/data_set/S0000000006/S0000000006.csv,./assets/data_set/S0000000006/S0000000006_land...
7,S0000000005,./assets/data_set/S0000000005/S0000000005.nii.gz,./assets/data_set/S0000000005/S0000000005.csv,./assets/data_set/S0000000005/S0000000005_land...


Split the dataset into train and test

In [10]:
test_size = 0.2
test_count = int(test_size * len(data_df))

# Shuffle the DataFrame
df_shuffled = data_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the DataFrame
train_df = df_shuffled.iloc[:-test_count]
test_df = df_shuffled.iloc[-test_count:]

In [11]:
train_df.to_csv(os.path.join(target_path, 'train.csv'), index=False)
test_df.to_csv(os.path.join(target_path, 'test.csv'), index=False)

In [12]:
import os

num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")


Number of CPU cores: 24
